In [1]:
import sys
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

In [2]:
sys.path.append('../')

In [3]:
from interpretability.sae.sae import SAE
from utils.inference_utils.llm import LLM
from utils.train_utils.train_sae import train_sae

In [4]:
model_name_or_path = 'Qwen/Qwen2-1.5B-Instruct'
device = 'cuda'

In [5]:
llm = LLM(
    model_name_or_path=model_name_or_path,
    device=device
) # Initialize a model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
for name, module in llm.model.named_modules():
    print(name) # choose which model's module activations you want to save into a train dataset


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [7]:
# Let it be `model.layers.27` for 'Qwen/Qwen2-1.5B-Instruct'

hidden_state = llm.get_hidden_state(
    input_text=['It raining cats and dogs', 'My mom likes reading'],
    layer_name='model.layers.27'
)

In [13]:
hidden_state.shape # Shape of the hidden_state

torch.Size([2, 1536])

## Load data for SAE training

In [14]:
from datasets import load_dataset

In [15]:
dataset = load_dataset(
    'fka/awesome-chatgpt-prompts'
)

In [16]:
prompts = dataset['train'].to_pandas()['prompt'].tolist()

In [17]:
prompts[:2]

['Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.',
 "Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google. Include every relevant heading possible. Keep the keyword density of the headings high. For each section of the outline, include the word count. Include FAQs section in the outline too, based on people also ask section from Google for the keyword. This outline must 

In [ ]:
activations_dataset = llm.get_hidden_state(
    input_text=prompts,
    layer_name='model.layers.27'
)

In [ ]:
activations_dataset = TensorDataset(activations_dataset)